In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, metrics
import multiprocessing
from gensim.models.word2vec import Word2Vec
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
from gensim.models.phrases import Phrases, Phraser
import nltk
print(cores)
from time import time
import tensorflow as tf
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
print(tf.__version__)

4
2.4.0


In [3]:
dataset = pd.read_csv('/home/aayush/task3_midas/datasets/imbalanced_dataset_200.csv')
train, test = model_selection.train_test_split(dataset, test_size=0.3)
x_train = train["pre_processed_text"].values
x_test = test["pre_processed_text"].values
y_train = train["category"].values
y_test = test["category"].values
print("our training set contains %d examples"%(len(x_train)))
print("our test set contains %d examples"%(len(x_test)))

our training set contains 11090 examples
our test set contains 4754 examples


In [4]:
w2v_model = Word2Vec(vector_size = 300,window=8, min_count=1, sg=1,  workers=cores-1)
word_sentences = list()
for x in x_train:
    word_sentences.append(x.split())
t = time()
w2v_model.build_vocab(word_sentences, progress_per=100)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
print(len(word_sentences))

Time to build vocab: 0.01 mins
11090


In [5]:
t = time()

w2v_model.train(word_sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


Time to train the model: 1.77 mins


In [6]:
w2v_model.wv.most_similar(positive=['kid'])

[('chestbio', 0.5711948275566101),
 ('greenmom', 0.5627349019050598),
 ('pyjamasetspecifications', 0.5609691143035889),
 ('chairorka', 0.5544642210006714),
 ('greymom', 0.5522589087486267),
 ('qp', 0.5348389148712158),
 ('comfortablebio', 0.5308143496513367),
 ('topattractive', 0.5308120250701904),
 ('durablebela', 0.5293307900428772),
 ('bagssuitcase', 0.5269551873207092)]

In [7]:
print(w2v_model.wv.key_to_index["car"])


49


In [8]:
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(x_train)
## padding sequence
X_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=2000, padding="post", truncating="post")


NameError: name 'tokenizer' is not defined

In [ ]:
lst_text2seq= tokenizer.texts_to_sequences(x_test)
## padding sequence
X_test = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=2000, padding="post", truncating="post")


In [ ]:

## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(w2v_model.wv.key_to_index)+1, 300))
for word,idx in w2v_model.wv.key_to_index.items():
    ## update the row with vector
    try:
        embeddings[idx] =  w2v_model.wv.get_vector(word)
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

In [ ]:
print(embeddings.shape)
print(X_train)

In [ ]:
## input
x_in = layers.Input(shape=(2000,))
## embedding
x = layers.Embedding(embeddings.shape[0],  
                     embeddings.shape[1],
                     weights=[embeddings],
                     input_length=2000, trainable=False)(x_in)

## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2, 
                         return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2))(x)
## final dense layers
x = layers.Dense(64, activation='relu')(x)
y_out = layers.Dense(3, activation='softmax')(x)
## compile
model = models.Model(x_in, y_out)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()